In [11]:
import praw
import csv
import pandas as pd
reddit = praw.Reddit(
    client_id="D8uhE8xekateXsNSjds3_Q",
    client_secret="RcAkITSHFpjagUK8GNwoHbiq95ko6g", 
    user_agent="Scraper")

In [12]:
sg_subreddit = reddit.subreddit("Singapore")
print(sg_subreddit.display_name)

Singapore


In [13]:
submission_headers = ['author', 'created_utc', 'id', 
                      'is_original_content', 'is_self', 
                      'link_flair_text', 'locked', 'name', 
                      'num_comments', 'over_18', 'permalink', 
                      'score', 'selftext', 'spoiler', 'stickied', 
                      'subreddit', 'title', 'upvote_ratio', 'url']


sub_headers = ['title', 'created_utc', 'id' ,'score','num_comments', 'upvote_ratio', 'permalink']

search_terms = ["lgbtq", "lgbt", "gay", "lesbian", "bisexual", "transgender", "queer", "pink dot", "pride month", "section 377A", "same-sex marriage", 'homosexual','sexual orientation', 'tranny', 'ladyboy', 'trans', 'anti-gay law', '377a penal code']

with open("reddit_test_submission_db.csv", 'a', 
          encoding="utf-8", newline='') as f_object:
    for term in search_terms: 
        posts = sg_subreddit.search(term)
        for post in posts:
            if post.score > 50: 
        # fields requested thourgh PRAW
                data = {'title': post.title, 'score': post.score, 'num_comments': post.num_comments,
                        'created_utc': post.created_utc, 'id': post.id ,
                        'upvote_ratio': post.upvote_ratio, 'permalink': post.permalink, 
                        }
                dictwriter_object = csv.DictWriter(
                    f_object, fieldnames=sub_headers)
                dictwriter_object.writerow(data)
    f_object.close()
    
# Delete duplicates
post_db = pd.read_csv("reddit_test_submission_db.csv", 
                      names=sub_headers, header=0)
post_db.drop_duplicates(subset="permalink", 
                        keep="last", inplace=True)
post_db.to_csv("reddit_test_submission_db.csv", 
               index=False, chunksize=1000)

In [14]:
# create submissions
comment_headers = ['author', 'body', 'created_utc', 
                   'distinguished', 'edited', 'id', 
                   'is_submitter', 'link_id', 'parent_id', 
                   'permalink', 'saved', 'score', 'stickied', 
                   'submission', 'subreddit', 'subreddit_id']
with open('reddit_test_comment_db.csv', 'a') as comment_file:
    comments_db = pd.read_csv('reddit_test_comment_db.csv', 
                              usecols=["submission"], 
                              names=comment_headers)
    comment_file.close()
submission_db = pd.read_csv("reddit_test_submission_db.csv", 
                            usecols=["created_utc", "id"])

comments_set = set(comments_db["submission"])
# 259,200 is three days worth of seconds
try:
    time_cutoff = max([created_utc for post_id, created_utc in 
         zip(submission_db.id, submission_db.created_utc) 
         if post_id in comments_set]) - 259200
except:
    time_cutoff = submission_db["created_utc"].min()
submissions_to_pull = submission_db.loc[submission_db
                                        ["created_utc"] >= 
                                        time_cutoff, "id"]
#This block pulls all comments for the list of submissions we have identified
with open("reddit_test_comment_DB.csv", 'a', 
          encoding="utf-8", newline='') as f_object:
    for row in submissions_to_pull:
        submission = reddit.submission(id=row)
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            data = {'author': comment.author, 'body': 
                    comment.body, 'created_utc': 
                    comment.created_utc, 'distinguished': 
                    comment.distinguished, 'edited': 
                    comment.edited, 'id': comment.id, 
                    'is_submitter': comment.is_submitter, 
                    'link_id': comment.link_id, 'parent_id': 
                    comment.parent_id, 'permalink': 
                    comment.permalink, 'saved': comment.saved, 
                    'score': comment.score, 'stickied': 
                    comment.stickied, 'submission': 
                    comment.submission, 'subreddit': 
                    comment.subreddit, 
                    'subreddit_id': comment.subreddit_id}
            dictwriter_object = csv.DictWriter(f_object, 
                                               fieldnames=
                                               comment_headers)
            dictwriter_object.writerow(data)
    f_object.close()
    
# drop duplicate rows
comment_db = pd.read_csv("reddit_test_comment_DB.csv", 
                         names=comment_headers, header=0)

comment_db.drop_duplicates(subset=["permalink", "body", "id"],
                           keep = "last", inplace=True)

comment_db.drop_duplicates(subset="permalink", 
                           keep = "first", inplace=True)
comment_db.to_csv("reddit_test_comment_DB.csv", 
                  index=False, chunksize=1000)

In [15]:
capturedData = pd.read_csv('reddit_test_comment_DB.csv')
capturedData.to_csv('capturedData23.csv', index=False) 